In [ ]:
import tensorflow as tf
import numpy as np
from os import path, getcwd, chdir
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [ ]:
#Generating models to classify which daughter vessel cells pick to enter 
import pandas as pd

df = pd.read_csv (r'G8FBCinfo.csv')
print (df)
Diameter=np.zeros((len(df)+1,1))
Q=np.zeros((len(df)+1,1))

Di=df['Diii']
volumeflow=df['vol_florate Ca=1']
for i in range(len(df)):
  Diameter[i+1,0]=Di[i]
  Q[i+1,0]=volumeflow[i]

BIF = pd.read_csv (r'bif_relation8.csv')
BIF1=BIF['mother']
BIF2=BIF['daughter1']
BIF3=BIF['daughter2'] 

mother=np.zeros((len(BIF1)+1,1))
daughter1=np.zeros((len(BIF1)+1,1))
daughter2=np.zeros((len(BIF1)+1,1))
for i in range(len(BIF1)):
  mother[i+1,0]=BIF1[i]
  daughter1[i+1,0]=BIF2[i]
  daughter2[i+1,0]=BIF3[i]


#READING BIF
#Reading DNS Data 
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

testbif=960001
filetest='960001'

store_modelc1=[]
store_acc_c1=[]
for iter in range(1,2):
  print('iter=',iter)
  if iter <1000: # iter!=(testbif-950000):
    biff=iter
    file='fort.'+str(960000+biff)
    
    f=open(file,"r")
    lines=f.readlines()
    X1=[]
    X2=[]
    X3=[]
    X4=[]

    Y1=[]
    Y2=[]
    Z1=[]
    T1=[]
    for x in lines:
        X1.append(float(x.split()[0]))
        X2.append(float(x.split()[1]))
        X3.append(float(x.split()[2]))
        X4.append(float(x.split()[3]))

        Y1.append(float(x.split()[4]))
        Y2.append(float(x.split()[5]))


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    df = pd.read_csv (r'G8FBCinfo.csv')
    print (df)
    Diameter=np.zeros((len(df)+1,1))
    Q=np.zeros((len(df)+1,1))

    Di=df['Diii']
    volumeflow=df['vol_florate Ca=1']
    for i in range(len(df)):
      Diameter[i+1,0]=Di[i]
      Q[i+1,0]=volumeflow[i]

    BIF = pd.read_csv (r'bif_relation8.csv')
    BIF1=BIF['mother']
    BIF2=BIF['daughter1']
    BIF3=BIF['daughter2'] 

    mother=np.zeros((len(BIF1)+1,1))
    daughter1=np.zeros((len(BIF1)+1,1))
    daughter2=np.zeros((len(BIF1)+1,1))
    for i in range(len(BIF1)):
      mother[i+1,0]=BIF1[i]
      daughter1[i+1,0]=BIF2[i]
      daughter2[i+1,0]=BIF3[i]

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    for i in range(len(BIF1)+1):
      if i==biff:
        print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
        mom=mother[i,0]
        dau1=daughter1[i,0]
        dau2=daughter2[i,0]

    #X, Y for the NN
    X_Data=np.zeros((int(len(X1)),8))
    Y_Data=np.zeros((int(len(X1)),2))
    for i in range(len(X1)):
      X_Data[i,0]=X3[i]/Diameter[int(mom),0]
      X_Data[i,1]=X4[i]
      if Diameter[int(dau1),0]<Diameter[int(dau2),0]:
        X_Data[i,2]=1
        X_Data[i,3]=0
      elif Diameter[int(dau1),0]>Diameter[int(dau2),0]:
        X_Data[i,2]=0
        X_Data[i,3]=1
      else:
        X_Data[i,2]=0
        X_Data[i,3]=0

    #volume flow rate 
    for i in range(len(X1)):
      X_Data[i,4]=Q[int(dau1),0]/Q[int(mom),0]
      X_Data[i,5]=Q[int(dau2),0]/Q[int(mom),0]

    #Diameter 
    for i in range(len(X1)):
      X_Data[i,6]=Diameter[int(dau1),0]/Diameter[int(mom),0]
      X_Data[i,7]=Diameter[int(dau2),0]/Diameter[int(mom),0]

    for i in range(len(X1)):
      if X2[i]==dau1:
        #print('i',i)
        Y_Data[i,0]=1
        Y_Data[i,1]=0
      elif X2[i]==dau2:
        #print('ii',i)
        Y_Data[i,0]=0
        Y_Data[i,1]=1

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Training and Test Data 
    x_train=X_Data[:int(0.8*len(X_Data)),:]
    y_train=Y_Data[:int(0.8*len(Y_Data)),:]
    x_test=X_Data[int(0.8*len(X_Data)):,:]
    y_test=Y_Data[int(0.8*len(Y_Data)):,:]

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>MODEL
    modelc1 = tf.keras.models.Sequential([
            tf.keras.layers.Dense(4,activation=tf.nn.relu, input_shape=(8,)),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(2, activation=tf.nn.softmax)
        ])
    modelc1.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    modelc1.summary()


    history=modelc1.fit(
                  x_train, y_train, epochs=1000
        )

    store_modelc1.append(modelc1)
    store_acc_c1.append(np.average(history.history['accuracy']))

#storing the models 
for i in range(0,number_bifurcation):
  store_modelc1[i].save('myy_model'+str(i)+'/dd') 

# tar the models 
!tar -czvf "/content/G8_full_class1.tar.gz" "/content/full_classification_c1"     #[run this cell to create tar.gz files]

In [ ]:
#generating the models for predicting how far cells are from the 
import pandas as pd

df = pd.read_csv (r'G8FBCinfo.csv')
print (df)
Diameter=np.zeros((len(df)+1,1))
Q=np.zeros((len(df)+1,1))

Di=df['Diii']
volumeflow=df['vol_florate Ca=1']
for i in range(len(df)):
  Diameter[i+1,0]=Di[i]
  Q[i+1,0]=volumeflow[i]

BIF = pd.read_csv (r'bif_relation8.csv')
BIF1=BIF['mother']
BIF2=BIF['daughter1']
BIF3=BIF['daughter2'] 

mother=np.zeros((len(BIF1)+1,1))
daughter1=np.zeros((len(BIF1)+1,1))
daughter2=np.zeros((len(BIF1)+1,1))
for i in range(len(BIF1)):
  mother[i+1,0]=BIF1[i]
  daughter1[i+1,0]=BIF2[i]
  daughter2[i+1,0]=BIF3[i]


#READING BIF
#Reading DNS Data 
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

record=[]
store_modelr1=[]
for iter in range(1,22):
  print('iter=',iter)
  if iter<100: #iter!=(testbif-950000):
    biff=iter
    file='fort.'+str(960000+biff)

    f=open(file,"r")
    lines=f.readlines()
    X1=[]
    X2=[]
    X3=[]
    X4=[]

    Y1=[]
    Y2=[]
    Z1=[]
    T1=[]
    for x in lines:
        X1.append(float(x.split()[0]))
        X2.append(float(x.split()[1]))
        X3.append(float(x.split()[2]))
        X4.append(float(x.split()[3]))

        Y1.append(float(x.split()[4]))
        Y2.append(float(x.split()[5]))


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    df = pd.read_csv (r'G8FBCinfo.csv')
    print (df)
    Diameter=np.zeros((len(df)+1,1))
    Q=np.zeros((len(df)+1,1))

    Di=df['Diii']
    volumeflow=df['vol_florate Ca=1']
    for i in range(len(df)):
      Diameter[i+1,0]=Di[i]
      Q[i+1,0]=volumeflow[i]

    BIF = pd.read_csv (r'bif_relation8.csv')
    BIF1=BIF['mother']
    BIF2=BIF['daughter1']
    BIF3=BIF['daughter2'] 

    mother=np.zeros((len(BIF1)+1,1))
    daughter1=np.zeros((len(BIF1)+1,1))
    daughter2=np.zeros((len(BIF1)+1,1))
    for i in range(len(BIF1)):
      mother[i+1,0]=BIF1[i]
      daughter1[i+1,0]=BIF2[i]
      daughter2[i+1,0]=BIF3[i]

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    for i in range(len(BIF1)+1):
      if i==biff:
        print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
        mom=mother[i,0]
        dau1=daughter1[i,0]
        dau2=daughter2[i,0]

    #X, Y for the NN
    X_Data=np.zeros((int(len(X1)),4))
    Y_Data=np.zeros((int(len(X1)),1))
    for i in range(len(X1)):
      X_Data[i,0]=X3[i]#/Diameter[int(mom),0]
      X_Data[i,1]=X4[i]

    for i in range(len(X1)):
      if X2[i]==dau1:
        X_Data[i,2]=1
      elif X2[i]==dau2:  
        X_Data[i,3]=0
    for i in range(len(X1)):
      
      if X2[i]==dau1:
        Y_Data[i,0] = Y1[i]#/Diameter[int(dau1),0]
      elif X2[i]==dau2:
        Y_Data[i,0] = Y1[i]#/Diameter[int(dau2),0]
    

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Training and Test Data 
    x_train=X_Data[:int(0.8*len(X_Data)),:]
    y_train=Y_Data[:int(0.8*len(Y_Data)),:]
    x_test=X_Data[int(0.8*len(X_Data)):,:]
    y_test=Y_Data[int(0.8*len(Y_Data)):,:]

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>MODEL
    modelr1 = tf.keras.models.Sequential([
            
            tf.keras.layers.Dense(10,activation=tf.nn.relu, input_shape=(4,)),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(40,activation=tf.nn.relu),
            tf.keras.layers.Dense(1, activation=tf.nn.relu)
        ])
    modelr1.compile(optimizer='adam',
                      loss='mse',
                      metrics=['mse'])

    history = modelr1.fit(
                  x_train, y_train, epochs=2000
                 
        )
    store_modelr1.append(modelr1)


for i in range(21):
  store_modelr1[i].save('new_full_regression/bif'+str(i+1)+'/my_model') 

!tar -czvf "/content/new_full_regression.tar.gz" "/content/new_full_regression"     #[run this cell to create tar.gz files]

In [ ]:
#classification for checking if cells are on which side of branch 
import pandas as pd

df = pd.read_csv (r'G9FBCinfo.csv')
print (df)
Diameter=np.zeros((len(df)+1,1))
Q=np.zeros((len(df)+1,1))

Di=df['D2']
volumeflow=df['vol_florate Ca=1']
for i in range(len(df)):
  Diameter[i+1,0]=Di[i]
  Q[i+1,0]=volumeflow[i]

BIF = pd.read_csv (r'bif_relation.csv')
BIF1=BIF['mother']
BIF2=BIF['daughter1']
BIF3=BIF['daughter2'] 

mother=np.zeros((len(BIF1)+1,1))
daughter1=np.zeros((len(BIF1)+1,1))
daughter2=np.zeros((len(BIF1)+1,1))
for i in range(len(BIF1)):
  mother[i+1,0]=BIF1[i]
  daughter1[i+1,0]=BIF2[i]
  daughter2[i+1,0]=BIF3[i]


#READING BIF
#Reading DNS Data 
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

record=[]
store_model2c=[]
store_acc=[]
for iter in range(1,21):
  print('iter=',iter)
  if iter <1000: # iter!=(testbif-950000):
    biff=iter

    file='fort.'+str(950000+biff)

    f=open(file,"r")
    lines=f.readlines()
    X1=[]
    X2=[]
    X3=[]
    X4=[]

    Y1=[]
    Y2=[]
    Z1=[]
    T1=[]
    for x in lines:
        X1.append(float(x.split()[0]))
        X2.append(float(x.split()[1]))
        X3.append(float(x.split()[2]))
        X4.append(float(x.split()[3]))

        Y1.append(float(x.split()[4]))
        Y2.append(float(x.split()[5]))


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    df = pd.read_csv (r'G9FBCinfo.csv')
    print (df)
    Diameter=np.zeros((len(df)+1,1))
    Q=np.zeros((len(df)+1,1))

    Di=df['D2']
    volumeflow=df['vol_florate Ca=1']
    for i in range(len(df)):
      Diameter[i+1,0]=Di[i]
      Q[i+1,0]=volumeflow[i]

    BIF = pd.read_csv (r'bif_relation.csv')
    BIF1=BIF['mother']
    BIF2=BIF['daughter1']
    BIF3=BIF['daughter2'] 

    mother=np.zeros((len(BIF1)+1,1))
    daughter1=np.zeros((len(BIF1)+1,1))
    daughter2=np.zeros((len(BIF1)+1,1))
    for i in range(len(BIF1)):
      mother[i+1,0]=BIF1[i]
      daughter1[i+1,0]=BIF2[i]
      daughter2[i+1,0]=BIF3[i]

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    for i in range(len(BIF1)+1):
      if i==biff:
        print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
        mom=mother[i,0]
        dau1=daughter1[i,0]
        dau2=daughter2[i,0]

    #X, Y for the NN
    X_Data=np.zeros((int(len(X1)),3))
    Y_Data=np.zeros((int(len(X1)),2))
    for i in range(len(X1)):
     
      X_Data[i,0]=X3[i]#/Diameter[int(mom),0]
      if X4[i]==1:
        X_Data[i,1] = 1
        X_Data[i,2] = 0
      elif X4[i]==-1:
        X_Data[i,1] = 0
        X_Data[i,2] = 1


   
    for i in range(len(X1)):
      if Y2[i]==1:
        #print('i',i)
        Y_Data[i,0]=1
        Y_Data[i,1]=0
      elif Y2[i]==-1:
        #print('ii',i)
        Y_Data[i,0]=0
        Y_Data[i,1]=1

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Training and Test Data 
    x_train=X_Data[:int(0.8*len(X_Data)),:]
    y_train=Y_Data[:int(0.8*len(Y_Data)),:]
    x_test=X_Data[int(0.8*len(X_Data)):,:]
    y_test=Y_Data[int(0.8*len(Y_Data)):,:]

    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>MODEL
    model2c = tf.keras.models.Sequential([

            tf.keras.layers.Dense(4,activation=tf.nn.relu, input_shape=(3,)),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(10,activation=tf.nn.relu),
            tf.keras.layers.Dense(2, activation=tf.nn.softmax)
        ])
    model2c.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

    model2c.summary()


    history=model2c.fit(
                  x_train, y_train, epochs=1000
        )
    store_model2c.append(model2c)
    store_acc.append(np.average(history.history['accuracy']))

for i in range(21):
  store_model2c[i].save('full_classification_c2/bif'+str(i+1)+'/my_model') 
  



In [ ]:
#read modelc1
!tar -xzvf "/content/full_class1.tar.gz"  #[run this cell to untar the files]
#untar regression 
!tar -xzvf "/content/new_full_regression.tar.gz"  #[run this cell to untar the files]
#read modelc1
!tar -xzvf "/content/full_class2.tar.gz"  #[run this cell to untar the files]

In [ ]:
models_c1=[]
for i in range(0,20):
  models_c1.append(tf.keras.models.load_model('content/full_classification_c1/bif'+str(i+1)+'/my_model'))

models_c2=[]
for i in range(0,20):
  models_c2.append(tf.keras.models.load_model('content/full_classification_c2/bif'+str(i+1)+'/my_model'))

models_r=[]
for i in range(0,20):
  models_r.append(tf.keras.models.load_model('content/new_full_regression/bif'+str(i+1)+'/my_model'))

In [ ]:
#reading DRS data for the RBCs at the entrance of the biggest artery 
import pandas as pd
import matplotlib.pyplot as plt
rbc=22#len(x_test)-1                       
x_classification=np.zeros((1,8))
x_regression=np.zeros((1,4))

file='fort.95001'
biff=1
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   INPUT DATA from vessel 1 >>>>>>>>>>>
df = pd.read_csv (r'G9FBCinfo.csv')
print (df)
Diameter=np.zeros((len(df)+1,1))
Q=np.zeros((len(df)+1,1))

Di=df['D2']
volumeflow=df['vol_florate Ca=1']
for i in range(len(df)):
  Diameter[i+1,0]=Di[i]
  Q[i+1,0]=volumeflow[i]

BIF = pd.read_csv (r'bif_relation.csv')
BIF1=BIF['mother']
BIF2=BIF['daughter1']
BIF3=BIF['daughter2'] 

mother=np.zeros((len(BIF1)+1,1))
daughter1=np.zeros((len(BIF1)+1,1))
daughter2=np.zeros((len(BIF1)+1,1))
for i in range(len(BIF1)):
  mother[i+1,0]=BIF1[i]
  daughter1[i+1,0]=BIF2[i]
  daughter2[i+1,0]=BIF3[i]

it=1


for it in range(1):
  
  print('biff',biff)

  f=open(file,"r")
  lines=f.readlines()
  X1=[]
  X2=[]
  X3=[]
  X4=[]

  Y1=[]
  Y2=[]
  Z1=[]
  T1=[]
  for x in lines:
      X1.append(float(x.split()[0]))
      X2.append(float(x.split()[1]))
      X3.append(float(x.split()[2]))
      X4.append(float(x.split()[3]))

      Y1.append(float(x.split()[4]))
      Y2.append(float(x.split()[5]))

  print('len(X1)',len(X1))
  #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  df = pd.read_csv (r'G9FBCinfo.csv')
  print (df)
  Diameter=np.zeros((len(df)+1,1))
  Q=np.zeros((len(df)+1,1))

  Di=df['D2']
  volumeflow=df['vol_florate Ca=1']
  for i in range(len(df)):
    Diameter[i+1,0]=Di[i]
    Q[i+1,0]=volumeflow[i]

  BIF = pd.read_csv (r'bif_relation.csv')
  BIF1=BIF['mother']
  BIF2=BIF['daughter1']
  BIF3=BIF['daughter2'] 

  mother=np.zeros((len(BIF1)+1,1))
  daughter1=np.zeros((len(BIF1)+1,1))
  daughter2=np.zeros((len(BIF1)+1,1))
  for i in range(len(BIF1)):
    mother[i+1,0]=BIF1[i]
    daughter1[i+1,0]=BIF2[i]
    daughter2[i+1,0]=BIF3[i]

  #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

  for i in range(len(BIF1)+1):
    if i==biff:
      print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
      mom=mother[i,0]
      dau1=daughter1[i,0]
      dau2=daughter2[i,0]
  print('mom',mom)
  #X, Y for the NN
  X_Data=np.zeros((int(len(X1)),8))
  Y_Data=np.zeros((int(len(X1)),2))
  for i in range(len(X1)):
    X_Data[i,0]=X3[i]/Diameter[int(mom),0]
    print(i,X3[i],int(mom),1)
    X_Data[i,1]=X4[i]
    if Diameter[int(dau1),0]<Diameter[int(dau2),0]:
      X_Data[i,2]=1
      X_Data[i,3]=0
    elif Diameter[int(dau1),0]>Diameter[int(dau2),0]:
      X_Data[i,2]=0
      X_Data[i,3]=1
    else:
      X_Data[i,2]=0
      X_Data[i,3]=0

  #volume flow rate 
  for i in range(len(X1)):
    X_Data[i,4]=Q[int(dau1),0]/Q[int(mom),0]
    X_Data[i,5]=Q[int(dau2),0]/Q[int(mom),0]

  #Diameter 
  for i in range(len(X1)):
    X_Data[i,6]=Diameter[int(dau1),0]/Diameter[int(mom),0]
    X_Data[i,7]=Diameter[int(dau2),0]/Diameter[int(mom),0]

  for i in range(len(X1)):
    if X2[i]==dau1:
      #print('i',i)
      Y_Data[i,0]=1
      Y_Data[i,1]=0
    elif X2[i]==dau2:
      #print('ii',i)
      Y_Data[i,0]=0
      Y_Data[i,1]=1

  #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>Training and Test Data 
  x_train=X_Data[:int(0.8*len(X_Data)),:]
  y_train=Y_Data[:int(0.8*len(Y_Data)),:]
  x_test=X_Data[int(0.8*len(X_Data)):,:]
  y_test=Y_Data[int(0.8*len(Y_Data)):,:]

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   INPUT DATA from vessel 1 <<<<<<<<<<<<




In [ ]:
#MAIN ---->start from vessel = 1, and sequantially predict the cells motion 

ALL_vessel=[]
ALL_bif=[]
ALL_dist=[]
vessel_enter = 1

bif_rbc=[]

NN=700
bif_histdist=np.zeros((22,NN))
bif_histrbc=np.zeros((22,NN))
bif_histuplow=np.zeros((22,NN,2))

Nstar_hist=[]
c_enter1=0
c_enter2=0

for bi in range(1,15):
  c_enter1=0
  c_enter2=0
  for rbc in range(1,NN):
    print('Working bifurcation =  ', bi,'rbc=',rbc)
    #bifhist=[]
    #vesselhist=[]
    #distancehist=[]
    #bifhist.append(15)
    #vesselhist.append(vessel_enter)

    for i in range(len(BIF1)+1):
        if i== bi:
          #print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
          mom=mother[i,0]
          dau1=daughter1[i,0]
          dau2=daughter2[i,0]

    if bi == 1:
      # Classification 1
      RRR=x_train[rbc:rbc+1]
      pre_c1=models_c1[bi-1].predict(RRR)       ###CAREFULE
      if pre_c1[0,0]>pre_c1[0,1]:
        vessel_enter=dau1
        c_enter1=c_enter1+1
      elif pre_c1[0,0]<pre_c1[0,1]:
        vessel_enter=dau2
        c_enter2=c_enter2+1
      for i in range(len(BIF1)+1):
        if mother[i,0]==vessel_enter:
          bif_enter = i    

  

      bif_histrbc[bif_enter,rbc] = 1
      ##############################
      #Regression 
      x_regression=np.zeros((1,4))
      x_regression[0,0] = RRR[0,0]*Diameter[int(mom),0]
      x_regression[0,1] = RRR[0,1]
      if dau1 ==vessel_enter:
        x_regression[0,2]=1
      elif dau2 ==vessel_enter:  
        x_regression[0,3]=0
      pre_r=models_r[bi-1].predict(x_regression)

      bif_histdist[bif_enter,rbc] = pre_r[0,0]
      #distancehist.append(pre_r[0,0])
      #print('check',bifhist[len(bifhist)-1]-1,'X_reg',pre_r[0,0],'x_regression',x_regression)
      ########################################################################################
      #Classification 2
      x_class = np.zeros((1,3))
      x_class[0,0]=RRR[0,0]*Diameter[int(mom),0]
    
      if RRR[0,1]==1:
        x_class[0,1] =1 
        x_class[0,2] =0
      if RRR[0,1]==-1:
        x_class[0,1] =0
        x_class[0,2] =1

      pre_c2=models_c2[bi-1].predict(x_class)
      if  pre_c2[0,0]>pre_c2[0,1]:
        x_class[0,1] =1 
        x_class[0,2] =0
      elif pre_c2[0,0]<pre_c2[0,1]:
        x_class[0,1] =0
        x_class[0,2] =1

      bif_histuplow[bif_enter,rbc,0]=x_class[0,1]
      bif_histuplow[bif_enter,rbc,1]=x_class[0,2]

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
      if x_train[rbc:rbc+1,1][0] == 1:
        bif_histuplow[bif_enter,rbc,0]=1
        bif_histuplow[bif_enter,rbc,1]=0
      elif x_train[rbc:rbc+1,1][0] == -1:
        bif_histuplow[bif_enter,rbc,0]=0
        bif_histuplow[bif_enter,rbc,1]=1        
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

      #if bi ==15:
      #  print('bif_enter',bif_enter,'before bif15',x_class,'after bif15',bif_histuplow[bif_enter,rbc,:])

    else :

      
      if bif_histrbc[bi,rbc] == 1:
        #Classification 1
        #print('ggg',bi)
        for i in range(len(BIF1)+1):
          if i==bi:
            #print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
            iter_mom=mother[i,0]
            iter_dau1=daughter1[i,0]
            iter_dau2=daughter2[i,0]
        

        x_classification = np.zeros((1,8))
        x_classification[0,0]=bif_histdist[bi,rbc]/Diameter[int(iter_mom),0]

        if  bif_histuplow[bi,rbc,0]>bif_histuplow[bi,rbc,1]:
          x_classification[0,1]=1
        elif bif_histuplow[bi,rbc,0]<bif_histuplow[bi,rbc,1]:
          x_classification[0,1]=-1
        
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        x_classification[0,0] = x_train[rbc:rbc+1][0,0]
        x_classification[0,1] = (-1)**randrange(10)#x_train[rbc:rbc+1][0,1]
        #@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        
        if Diameter[int(iter_dau1),0]<Diameter[int(iter_dau2),0]:
          x_classification[0,2]=1
          x_classification[0,3]=0
        elif Diameter[int(iter_dau1),0]>Diameter[int(iter_dau2),0]:
          x_classification[0,2]=0
          x_classification[0,3]=1
        else:
          x_classification[0,2]=0
          x_classification[0,3]=0

        x_classification[0,4]=Q[int(iter_dau1),0]/Q[int(iter_mom),0]
        x_classification[0,5]=Q[int(iter_dau2),0]/Q[int(iter_mom),0]

        x_classification[0,6]=Diameter[int(iter_dau1),0]/Diameter[int(iter_mom),0]
        x_classification[0,7]=Diameter[int(iter_dau2),0]/Diameter[int(iter_mom),0]

          #print('it',it,'bifhist',bifhist,'fff',bifhist[len(bifhist)-1]-1)
        pre_c1=models_c1[bi-1].predict(x_classification)
        if pre_c1[0,0]>pre_c1[0,1]:
          vessel_enter=dau1
          c_enter1=c_enter1+1
        elif pre_c1[0,0]<pre_c1[0,1]:
          vessel_enter=dau2
          c_enter2=c_enter2+1

        if bi == 16:
          print('c1',x_classification,'x_train[rbc:rbc+1][0,0]',x_train[rbc:rbc+1][0,0]*Diameter[30,0],'vessel_enter',vessel_enter)

        #if bifhist[len(bifhist)-1]==18:
        #  print('bif18',pre_r[0,0],'x_regression',x_regression)
        #  hhy.append(pre_r[0,0])
        #  hhx.append(x_regression)

        gg1 = 0
        for i in range(len(BIF1)+1):
          if mother[i,0]==vessel_enter:
            bif_enter = i  
            gg1=1  

        #if bi ==18:
        #  print('x_classification',x_classification)

        #if bi==18:
        #  print('bif_enter',bif_enter,'vessel_enter',vessel_enter)
        if gg1 ==1:
          bif_histrbc[bif_enter,rbc] = 1

        #Regression 
        
        x_regression=np.zeros((1,4))
        x_regression[0,0] = bif_histdist[bi,rbc]

        if  bif_histuplow[bi,rbc,0]>bif_histuplow[bi,rbc,1]:
          x_regression[0,1] =1
        elif bif_histuplow[bi,rbc,0]<bif_histuplow[bi,rbc,1]:
          x_regression[0,1] =-1
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        x_regression[0,0] = x_train[rbc:rbc+1][0,0]*Diameter[int(iter_mom),0]
        x_regression[0,1] = x_train[rbc:rbc+1][0,1]
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    

        if iter_dau1 ==vessel_enter:
          x_regression[0,2]=1
        elif iter_dau2 ==vessel_enter:  
          x_regression[0,3]=0

        pre_r=models_r[bi-1].predict(x_regression)
        bif_histdist[bif_enter,rbc] = pre_r[0,0]
      
        #Classification 
        x_class[0,0] = bif_histdist[bi,rbc]

        
        if   bif_histuplow[bi,rbc,0]> bif_histuplow[bi,rbc,1]:
          x_class[0,1] =1 
          x_class[0,2] =0
        elif  bif_histuplow[bi,rbc,0]< bif_histuplow[bi,rbc,1]:
          x_class[0,1] =0
          x_class[0,2] =1

        #if bi ==16:
        #  print('bif_histdist[bi,rbc]',bif_histdist[bi,rbc],'bif_histuplow[bi,rbc,0]',bif_histuplow[bi,rbc,0],'bif_histuplow[bi,rbc,1]',bif_histuplow[bi,rbc,1])
        #  print('x_class',x_class)

        pre_c2=models_c2[bi-1].predict(x_class)
        if  pre_c2[0,0]>pre_c2[0,1]:
          x_class[0,1] =1 
          x_class[0,2] =0
        elif pre_c2[0,0]<pre_c2[0,1]:
          x_class[0,1] =0
          x_class[0,2] =1

        #if bi ==16:
        #  print('x_class after models_c2',x_class)


        

        bif_histuplow[bif_enter,rbc,0]=x_class[0,1]
        bif_histuplow[bif_enter,rbc,1]=x_class[0,2]

#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        if x_train[rbc:rbc+1,1][0] == 1:
          bif_histuplow[bif_enter,rbc,0]=1
          bif_histuplow[bif_enter,rbc,1]=0
        elif x_train[rbc:rbc+1,1][0] == -1:
          bif_histuplow[bif_enter,rbc,0]=0
          bif_histuplow[bif_enter,rbc,1]=1        
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ 

  if c_enter1/(c_enter1+c_enter2) > c_enter2/(c_enter1+c_enter2):
    Nstar_hist.append([bi,c_enter1/(c_enter1+c_enter2)])  
  else:
    Nstar_hist.append([bi,c_enter2/(c_enter1+c_enter2)])
        #if bi ==16:
        #  print('beforebif16',x_class,'after bif 16',bif_histuplow[bif_enter,rbc,:])
###ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ


In [ ]:
#calculation of Ht based on determined N_star

Nstar=[]
for i in range(number_ bif):
  Nstar.append(Nstar_hist[i][1])

Ht=np.zeros((68,1))
Ht[30,0]=0.305  # hematocirt you specify at the biggest artery
for bi in range(15,21):
  for i in range(len(BIF1)+1):
    if i== bi:
      #print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
      mom=mother[i,0]
      dau1=daughter1[i,0]
      dau2=daughter2[i,0]

  if Q[int(dau1),0]>Q[int(dau2),0]:
    Ht[int(dau1),0]=(Q[int(mom),0]/Q[int(dau1),0])*Ht[int(mom),0]*Nstar[bi-1]
    Ht[int(dau2),0]=(Q[int(mom),0]/Q[int(dau2),0])*Ht[int(mom),0]*(1-Nstar[bi-1])
  elif Q[int(dau1),0]<Q[int(dau2),0]:
    Ht[int(dau2),0]=(Q[int(mom),0]/Q[int(dau2),0])*Ht[int(mom),0]*Nstar[bi-1]
    Ht[int(dau1),0]=(Q[int(mom),0]/Q[int(dau1),0])*Ht[int(mom),0]*(1-Nstar[bi-1])

Ht[1,0]=0.22   # Hematocirt you specify at the biggest artery  
for bi in range(1,15):
  for i in range(len(BIF1)+1):
    if i== bi:
      #print('mother',mother[i,0], 'daugther1,2',daughter1[i,0],daughter2[i,0])
      mom=mother[i,0]
      dau1=daughter1[i,0]
      dau2=daughter2[i,0]

  if Q[int(dau1),0]>Q[int(dau2),0]:
    Ht[int(dau1),0]=(Q[int(mom),0]/Q[int(dau1),0])*Ht[int(mom),0]*Nstar[bi-1]
    Ht[int(dau2),0]=(Q[int(mom),0]/Q[int(dau2),0])*Ht[int(mom),0]*(1-Nstar[bi-1])
  elif Q[int(dau1),0]<Q[int(dau2),0]:
    Ht[int(dau2),0]=(Q[int(mom),0]/Q[int(dau2),0])*Ht[int(mom),0]*Nstar[bi-1]
    Ht[int(dau1),0]=(Q[int(mom),0]/Q[int(dau1),0])*Ht[int(mom),0]*(1-Nstar[bi-1])


# connectivity between vessel (We consider a index for each vessel)
Vessel_connect=[[48,4,22],
[49,6,23],
[50,12,49],
[51,13,26],
[52,20,48],
[53,45,51],
[54,31,36],
[55,33,41],
[56,37,38],
[57,39,47],
[58,55,47],
[59,29,44],
[60,28,59],
[61,46,60],
[62,27,61],
[63,58,62],
[64,53,63],
[65,57,64],
[66,52,65]]

for i in range(48,67):
  s=Vessel_connect[i-48][1]
  t=Vessel_connect[i-48][2]
  Ht[i,0]=((Q[s,0]/Q[i,0])*Ht[s,0])+((Q[t,0]/Q[i,0])*Ht[t,0])



# DSR data for the hematocirt 
DNS_Ht=np.array([0,0.221863691,
0.263893442,
0.304219081,
0.144445533,
0.316116461,
0.368161852,
0.273679597,
0.265546203,
0.524514399,
0.39202068,
0.260545728,
0.223081916,
0.219644394,
0.240361371,
0.214919409,
0.243003188,
0.307760499,
0.23868784,
0.179284078,
0.35816178,
0.292193356,
8.42E-02,
0.180797655,
0.272382993,
0.29086038,
0.245412158,
0.240453193,
0.269290385,
0.27483286,
0.305763674,
0.204788582,
0.38129298,
0.177499895,
0.264182526,
0.26420261,
0.233025774,
0.2677595,
0.13637168,
0.19094977,
0.192711064,
0.242118845,
0.198498037,
0.270364523,
0.108422717,
0.236125186,
0.34618363,
0.224914537,
9.28E-02,
0.267901598,
0.295417232,
0.205905192,
0.246336331,
0.327739376,
0.551403661,
0.171715337,
0.186282377,
0.233564211,
0.21539762,
0.216441201,
0.220805361,
0.245622445,
0.245051091,
0.219867168,
0.209732896,
0.211878377,
0.226368959,
0.282315975
])


DNS_X=[]
ML_Y=[]
for i in range(67):
  if Ht[i,0] !=0 and Ht[i,0]<0.4:
    DNS_X.append(DNS_Ht[i])
    ML_Y.append(Ht[i,0])
    print('******')
plt.scatter(DNS_X,ML_Y)
print(len(DNS_X))

Ht_ML=Ht